In [19]:
import os
import pandas as pd
import numpy as np

In [20]:
file = "/home/galaxy/data/QTL/pQTL/pQTL_2.txt"
result_dir = "/home/galaxy/data/QTL/pQTL"

In [21]:
df = pd.read_table(file, sep="\t")
df.head()

,QTL_ID,QTL_type.cis_or_trans.,Chromosome,Start_position,End_position,Target_Protein,Target_fullname,Uniprot_ID,Species,Pubmed,Gene_Name,Ensembl_Gene_ID,Gene_Type,Rs_ID,SNP_position,Build
0,NaN,NaN,1,NaN,NaN,NaN,Activin receptor type-1B,P36896,Homo sapiens,28240269,ACVR1B,ENSG00000135503,protein_coding,rs260507,2192639,GRCh37
1,NaN,NaN,1,NaN,NaN,NaN,40S ribosomal protein S7,P62081,Homo sapiens,28240269,RPS7,ENSG00000171863,protein_coding,rs11260698,5549525,GRCh37
2,NaN,NaN,1,NaN,NaN,NaN,Fetuin-B,Q9UGM5,Homo sapiens,28240269,FETUB,ENSG00000090512,protein_coding,rs228689,7879168,GRCh37
3,NaN,NaN,1,NaN,NaN,NaN,Mitogen-activated protein kinase 9,P45984,Homo sapiens,28240269,MAPK9,ENSG00000050748,protein_coding,rs2859387,7887248,GRCh37
4,NaN,NaN,1,NaN,NaN,NaN,Carbonic anhydrase 6,P23280,Homo sapiens,28240269,CA6,ENSG00000131686,protein_coding,rs12136689,8601118,GRCh37


In [22]:
df_sub = df[["Uniprot_ID", "Ensembl_Gene_ID", "Rs_ID", "Pubmed"]]
df_sub = df_sub[(df_sub["Uniprot_ID"] != "not found") & (df_sub["Ensembl_Gene_ID"] != "not found")]
df_sub.head()
# df_sub[["Rs_ID"]].drop_duplicates().to_csv("%s/rs_id.txt" % result_dir, sep="\t", header=False, index=False)

,Uniprot_ID,Ensembl_Gene_ID,Rs_ID,Pubmed
0,P36896,ENSG00000135503,rs260507,28240269
1,P62081,ENSG00000171863,rs11260698,28240269
2,Q9UGM5,ENSG00000090512,rs228689,28240269
3,P45984,ENSG00000050748,rs2859387,28240269
4,P23280,ENSG00000131686,rs12136689,28240269


In [8]:
# snp_pos = "/home/galaxy/data/QTL/pQTL/id_and_position_GRCh38.txt"
# df_pos = pd.read_table(snp_pos, sep="\t")
# df_pos.head()

In [9]:
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh38_common_all_20180418.vcf"
df_vcf = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df_vcf.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df_vcf_sub = df_vcf[["CHROM", "POS", "ID", "REF", "ALT"]]
df_merge = df_sub.merge(df_vcf_sub, how="left", left_on="Rs_ID", right_on="ID").dropna().drop_duplicates()
print(df_merge.head())
df_merge.to_csv("%s/info.txt" % result_dir, sep="\t", index=False)

  Uniprot_ID  Ensembl_Gene_ID       Rs_ID    Pubmed CHROM        POS  \
0     P36896  ENSG00000135503    rs260507  28240269     1  2261200.0   
1     P62081  ENSG00000171863  rs11260698  28240269     1  5489465.0   
2     Q9UGM5  ENSG00000090512    rs228689  28240269     1  7819108.0   
3     P45984  ENSG00000050748   rs2859387  28240269     1  7827188.0   
4     P23280  ENSG00000131686  rs12136689  28240269     1  8541059.0   

           ID REF    ALT  
0    rs260507   G  A,C,T  
1  rs11260698   C      T  
2    rs228689   G    A,C  
3   rs2859387   G    A,C  
4  rs12136689   A      C  


In [24]:
# convert into bed
####################
in_file = "/home/galaxy/data/QTL/pQTL/info.txt"
result_file = "/home/galaxy/data/QTL/pQTL/GRCh38_pQTL.bed"
##################
df = pd.read_table(in_file, sep="\t")
df["info"] = df["Uniprot_ID"] + ";" + df["Ensembl_Gene_ID"] + ";" + df["Pubmed"].astype(str) + ";" + df["Rs_ID"] + ";" + df["REF"] + ";" + df["ALT"]
df["chr_name"] = "chr" + df["CHROM"].astype(str)
df["start"] = df["POS"].astype(int) - 1
df["end"] = df["POS"].astype(int)
df_final = df[["chr_name", "start", "end", "info"]]
# df_final = df_final[""]
df_final.to_csv(result_file, sep="\t", index=False) # header=False, 